In [1]:
############################################################################
##
## Copyright (C) 2021 NVIDIA Corporation.  All rights reserved.
##
## NVIDIA Sample Code
##
## Please refer to the NVIDIA end user license agreement (EULA) associated
## with this source code for terms and conditions that govern your use of
## this software. Any use, reproduction, disclosure, or distribution of
## this software and related documentation outside the terms of the EULA
## is strictly prohibited.
##
############################################################################

In [2]:
###### https://degravek.github.io/project-pages/project1/2016/11/12/New-Notebook/ #################
# with modifications and GPU acceleration by Mark J Bennett mbennett@nvidia.com and Emanuel Scoullos escoullos@nvidia.com

###### Best when compared to:
###### https://www.bankofengland.co.uk/-/media/boe/files/working-paper/2019/
######        machine-learning-explainability-in-finance-an-application-to-default-risk-analysis.pdf

from collections import OrderedDict
import gc
import glob
import os
import pickle
import re
import sys
import time

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# make sure the path below points to the data directory where the FannieMae data was downloaded.
data_path = '/rapids/data'

n_gpus = 1
isPlot = False

if not n_gpus:
    from pandas import read_csv, concat, merge, DataFrame, get_dummies, Series, isnull
    from sklearn.preprocessing import OneHotEncoder
    from numpy import zeros, double, int8, concatenate
else:
    from cudf import read_csv, concat, merge, DataFrame, get_dummies, Series
    from cuml.preprocessing import OneHotEncoder
    from cupy import zeros,double, int8, concatenate

# Read and Process Fannie Mae Dataset from 2007-2012

In [3]:
!ls /rapids/data/*

/rapids/data/mortgage_2000-2016.tgz  /rapids/data/rmback.sh
/rapids/data/names.csv		     /rapids/data/rmfront.sh

/rapids/data/acq:
Acquisition_2007Q1.txt	Acquisition_2009Q1.txt	Acquisition_2011Q1.txt
Acquisition_2007Q2.txt	Acquisition_2009Q2.txt	Acquisition_2011Q2.txt
Acquisition_2007Q3.txt	Acquisition_2009Q3.txt	Acquisition_2011Q3.txt
Acquisition_2007Q4.txt	Acquisition_2009Q4.txt	Acquisition_2011Q4.txt
Acquisition_2008Q1.txt	Acquisition_2010Q1.txt	Acquisition_2012Q1.txt
Acquisition_2008Q2.txt	Acquisition_2010Q2.txt	Acquisition_2012Q2.txt
Acquisition_2008Q3.txt	Acquisition_2010Q3.txt	Acquisition_2012Q3.txt
Acquisition_2008Q4.txt	Acquisition_2010Q4.txt	Acquisition_2012Q4.txt

/rapids/data/perf:
Performance_2007Q1.txt	  Performance_2009Q2.txt_1  Performance_2011Q3.txt
Performance_2007Q2.txt	  Performance_2009Q3.txt_0  Performance_2011Q4.txt_0
Performance_2007Q3.txt	  Performance_2009Q3.txt_1  Performance_2011Q4.txt_1
Performance_2007Q4.txt	  Performance_2009Q4.txt    Performance_2012Q1.

In [4]:
col_acq = ['LoanID', 'Channel', 'SellerName', 'OrInterestRate', 'OrUnpaidPrinc', 'OrLoanTerm',
           'OrDate', 'FirstPayment', 'OrLTV', 'OrCLTV', 'NumBorrow', 'DTIRat', 'CreditScore',
           'FTHomeBuyer', 'LoanPurpose', 'PropertyType', 'NumUnits', 'OccStatus', 'PropertyState',
           'Zip', 'MortInsPerc', 'ProductType', 'CoCreditScore', 'Extra', 'MortInsType', 'RelMortInd'];
dtypesDict = OrderedDict([
    ("LoanID", "int64"),
    ("Channel", "category"),
    ("SellerName", "category"),
    ("OrInterestRate", "float64"),
    ("OrUnpaidPrinc", "int64"),
    ("OrLoanTerm", "int64"),
    ("OrDate", "str"),
    ("FirstPayment", "str"),
    ("OrLTV", "int64"),  # was:float64
    ("OrCLTV", "float64"),
    ("NumBorrow", "float64"),
    ("DTIRat", "float64"),
    ("CreditScore", "float64"),
    ("FTHomeBuyer", "category"),
    ("LoanPurpose", "category"),
    ("PropertyType", "category"),
    ("NumUnits", "int64"),
    ("OccStatus", "category"),
    ("PropertyState", "category"),
    ("Zip", "int64"),
    ("MortInsPerc", "float64"),
    ("ProductType", "category"),
    ("CoCreditScore", "float64"),
    ("Extra", "int64"),
    ("MortInsType", "category"),
    ("RelMortInd", "category"),
])

dtype = list(dtypesDict.values())
print(dtype)
print('col_acq list is len', len(col_acq))

['int64', 'category', 'category', 'float64', 'int64', 'int64', 'str', 'str', 'int64', 'float64', 'float64', 'float64', 'float64', 'category', 'category', 'category', 'int64', 'category', 'category', 'int64', 'float64', 'category', 'float64', 'int64', 'category', 'category']
col_acq list is len 26


In [5]:
# open Acquisition files from each quarter 2007 - 2012

pattern = r'Acquisition_(20[0][7,8,9]|201[0,1,2])Q[1-4].txt'

fpaths = sorted(filter(re.compile(pattern).match, os.listdir(f'{data_path}/acq')))

df_acq = concat([read_csv(f'{data_path}/acq/{fp}', sep='|', names=col_acq, index_col=False) 
                 for fp in fpaths], axis=0).reset_index(drop=True)
print('\n'.join(fpaths))

df_acq.head()

Acquisition_2007Q1.txt
Acquisition_2007Q2.txt
Acquisition_2007Q3.txt
Acquisition_2007Q4.txt
Acquisition_2008Q1.txt
Acquisition_2008Q2.txt
Acquisition_2008Q3.txt
Acquisition_2008Q4.txt
Acquisition_2009Q1.txt
Acquisition_2009Q2.txt
Acquisition_2009Q3.txt
Acquisition_2009Q4.txt
Acquisition_2010Q1.txt
Acquisition_2010Q2.txt
Acquisition_2010Q3.txt
Acquisition_2010Q4.txt
Acquisition_2011Q1.txt
Acquisition_2011Q2.txt
Acquisition_2011Q3.txt
Acquisition_2011Q4.txt
Acquisition_2012Q1.txt
Acquisition_2012Q2.txt
Acquisition_2012Q3.txt
Acquisition_2012Q4.txt


,LoanID,Channel,SellerName,OrInterestRate,OrUnpaidPrinc,OrLoanTerm,OrDate,FirstPayment,OrLTV,OrCLTV,NumBorrow,DTIRat,CreditScore,FTHomeBuyer,LoanPurpose,PropertyType,NumUnits,OccStatus,PropertyState,Zip,MortInsPerc,ProductType,CoCreditScore,Extra,MortInsType,RelMortInd
0,100001461640,R,"PNC BANK, N.A.",6.250,137000,360,01/2007,03/2007,56,56.0,2.0,37.0,741.0,N,C,SF,1,P,MI,486,<NA>,FRM,734.0,<NA>,N,200701
1,100015135004,R,SUNTRUST MORTGAGE INC.,6.000,116000,360,02/2007,04/2007,80,80.0,2.0,11.0,796.0,N,R,SF,1,S,GA,302,<NA>,FRM,762.0,<NA>,N,200701
2,100015306566,C,"CITIMORTGAGE, INC.",6.375,58000,180,02/2007,03/2007,78,78.0,2.0,30.0,710.0,N,R,SF,1,P,IN,465,<NA>,FRM,<NA>,<NA>,N,200701
3,100015319835,C,"BANK OF AMERICA, N.A.",6.125,353000,360,12/2006,02/2007,80,80.0,2.0,28.0,778.0,N,R,SF,1,P,MA,21,<NA>,FRM,656.0,<NA>,N,200701
4,100030521552,C,"GMAC MORTGAGE, LLC",5.875,385000,360,12/2006,03/2007,70,70.0,2.0,50.0,720.0,N,C,SF,1,P,CA,917,<NA>,FRM,700.0,<NA>,N,200701


In [6]:
col_per = ['LoanID', 'MonthRep', 'Servicer', 'CurrInterestRate', 'CAUPB', 'LoanAge', 'MonthsToMaturity',
           'AdMonthsToMaturity', 'MaturityDate', 'MSA', 'CLDS', 'ModFlag', 'ZeroBalCode', 'ZeroBalDate',
           'LastInstallDate', 'ForeclosureDate', 'DispositionDate', 'ForeclosureCosts', 'PPRC', 'AssetRecCost', 'MHRC',
           'ATFHP', 'NetSaleProceeds', 'CreditEnhProceeds', 'RPMWP', 'OFP', 'NIBUPB', 'PFUPB', 'RMWPF',
           'FPWA', 'ServicingIndicator']
print('col_per list is len',len(col_per),'but using only 2!')
start = time.time()

df_per = concat([read_csv(fp, sep='|', names=col_per, usecols=[0, 10],
                          dtype={'LoanID': int, 'CLDS': str}, index_col=False) for
                 fp in sorted(glob.glob(f'{data_path}/perf/*.txt*'))], axis=0)

print(time.time()-start,'secs')
tread = time.time()-start
print(df_per.shape)
df_per.loc[df_per['LoanID'] == 100006270289].tail(10) #,100960628290]].tail(10)

col_per list is len 31 but using only 2!
12.216453552246094 secs
(585303356, 2)


,LoanID,CLDS
130,100006270289,0
131,100006270289,-1
132,100006270289,0
133,100006270289,0
134,100006270289,0
135,100006270289,0
136,100006270289,1
137,100006270289,2
138,100006270289,3
139,100006270289,4


In [7]:
print(df_per.shape)
df_per = df_per.dropna(subset=['CLDS'])
df_per['CLDS'] = df_per['CLDS'].astype(str)
print(df_per.shape)
df_per.loc[df_per['LoanID'] == 100006270289].tail(10) #100960628290].tail(10)

(585303356, 2)
(585303356, 2)


,LoanID,CLDS
130,100006270289,0
131,100006270289,-1
132,100006270289,0
133,100006270289,0
134,100006270289,0
135,100006270289,0
136,100006270289,1
137,100006270289,2
138,100006270289,3
139,100006270289,4


  ## Loan delinquency

In [8]:

### SET TO 4 MONTHS (str) below.###
NUM_MONTHS = '4'
###################################


df_delinq4 = df_per.loc[df_per['CLDS'] == NUM_MONTHS]

print(df_delinq4.shape)
df_delinq4.drop_duplicates(subset='LoanID', keep='last', inplace=True)
print(df_delinq4.shape)
df_delinq4.loc[df_delinq4['LoanID'] == 100006270289].tail(10)

(724491, 2)
(432236, 2)


,LoanID,CLDS
139,100006270289,4


In [9]:
print(df_acq.shape, df_delinq4.shape)

start = time.time()
df = merge(df_acq, df_delinq4, on='LoanID', how='outer')
df['Zip'] = df['Zip'].astype(str)
print(time.time()-start, 'secs')

del df_acq, df_per, df_delinq4
gc.collect()

df = df.reset_index().rename(columns={'CLDS': 'Default'})
print(df.shape)

df.loc[df['Default'] == '4', 'Default'] = 1
df.loc[df['Default'].isnull(), 'Default'] = 0  # for null caused by outer join

df['Default'] = df['Default'].astype(int)
df.loc[df['LoanID'] == 100006270289].tail(10)  # Yes missed payments and so w/100960628290

(11232359, 26) (432236, 2)
0.09046173095703125 secs
(11232359, 28)


,index,LoanID,Channel,SellerName,OrInterestRate,OrUnpaidPrinc,OrLoanTerm,OrDate,FirstPayment,OrLTV,OrCLTV,NumBorrow,DTIRat,CreditScore,FTHomeBuyer,LoanPurpose,PropertyType,NumUnits,OccStatus,PropertyState,Zip,MortInsPerc,ProductType,CoCreditScore,Extra,MortInsType,RelMortInd,Default
1242081,1242081,100006270289,C,"WELLS FARGO BANK, N.A.",7.5,226000,360,11/2007,01/2008,95,95.0,1.0,42.0,704.0,Y,P,SF,1,P,CA,933,30.0,FRM,<NA>,1.0,N,200801,1


In [10]:
df['Default'].value_counts()
#Test no default  case:
df.loc[df['LoanID'] == 100010079393]

,index,LoanID,Channel,SellerName,OrInterestRate,OrUnpaidPrinc,OrLoanTerm,OrDate,FirstPayment,OrLTV,OrCLTV,NumBorrow,DTIRat,CreditScore,FTHomeBuyer,LoanPurpose,PropertyType,NumUnits,OccStatus,PropertyState,Zip,MortInsPerc,ProductType,CoCreditScore,Extra,MortInsType,RelMortInd,Default
5098609,5098609,100010079393,C,"WELLS FARGO BANK, N.A.",4.875,284000,360,01/2010,03/2010,80,80.0,1.0,32.0,773.0,Y,P,PU,1,P,TX,787,<NA>,FRM,<NA>,<NA>,N,201001,0


In [11]:
df.loc[df['Default'] == 1].head(10)  # focus on rearer defaults

,index,LoanID,Channel,SellerName,OrInterestRate,OrUnpaidPrinc,OrLoanTerm,OrDate,FirstPayment,OrLTV,OrCLTV,NumBorrow,DTIRat,CreditScore,FTHomeBuyer,LoanPurpose,PropertyType,NumUnits,OccStatus,PropertyState,Zip,MortInsPerc,ProductType,CoCreditScore,Extra,MortInsType,RelMortInd,Default
160,160,189527725762,R,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",6.125,270000,360,01/2007,03/2007,58,86.0,2.0,34.0,680.0,N,C,PU,1,P,SC,294,<NA>,FRM,<NA>,<NA>,N,200701,1
161,161,189532228559,R,OTHER,6.875,109000,360,03/2007,05/2007,80,80.0,2.0,50.0,629.0,N,R,SF,1,P,PA,161,<NA>,FRM,668.0,<NA>,N,200701,1
162,162,189534717140,C,"CITIMORTGAGE, INC.",6.000,190000,360,01/2007,03/2007,76,76.0,1.0,36.0,710.0,N,R,SF,1,P,MN,554,<NA>,FRM,<NA>,<NA>,N,200701,1
192,192,229620456178,C,"GMAC MORTGAGE, LLC",5.875,303000,360,01/2007,03/2007,51,71.0,1.0,51.0,736.0,N,C,SF,1,P,FL,331,<NA>,FRM,<NA>,<NA>,N,200701,1
193,193,229650823232,C,"BANK OF AMERICA, N.A.",7.000,188000,360,01/2007,03/2007,95,95.0,1.0,30.0,650.0,Y,P,CO,1,P,IL,600,30.0,FRM,<NA>,1.0,N,200701,1
224,224,118097796777,R,"GMAC MORTGAGE, LLC",6.500,250000,360,01/2007,03/2007,80,80.0,2.0,58.0,768.0,N,C,SF,1,P,AZ,864,<NA>,FRM,752.0,<NA>,N,200701,1
225,225,118141013687,C,"CITIMORTGAGE, INC.",6.500,250000,360,01/2007,03/2007,80,95.0,2.0,29.0,676.0,N,P,PU,1,P,CA,925,<NA>,FRM,<NA>,<NA>,N,200701,1
256,256,374304413957,B,"BANK OF AMERICA, N.A.",6.375,401000,360,12/2006,02/2007,90,90.0,1.0,51.0,738.0,N,R,SF,1,P,IL,604,25.0,FRM,<NA>,2.0,N,200701,1
257,257,374314490698,C,"BANK OF AMERICA, N.A.",7.250,71000,360,02/2007,03/2007,84,84.0,2.0,40.0,637.0,N,C,SF,1,I,NE,683,12.0,FRM,713.0,1.0,N,200701,1
258,258,374320005729,R,"BANK OF AMERICA, N.A.",6.375,223000,360,02/2007,04/2007,78,78.0,2.0,51.0,641.0,N,C,SF,1,P,TX,786,<NA>,FRM,662.0,<NA>,N,200701,1


Before filling in missing values, let’s first take a quick look at the distribution of values in several of the data columns. We can start with our target variable, Default.


In [12]:
nullHistogram = None
if n_gpus == 0:
    nullHistogram = df.apply(lambda x: x.isnull().sum(), axis=0)
nullHistogram

In [13]:
if n_gpus == 0 and isPlot:
    sns.countplot(df['Default'])
    vc = df['Default'].value_counts()  # Let's see the imbalance 
    print('default density = ', vc[1]/vc[0], "or 1 in", np.round(vc[0]/vc[1],1))
    vc

In [14]:
if n_gpus == 0 and isPlot:
    columns = ['OrInterestRate','OrCLTV','DTIRat','CreditScore','OrUnpaidPrinc']
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(8,7))
    plt.tight_layout(w_pad=3.0, h_pad=3.0)

    print(range(1, len(columns)+1), columns)
    for i, column in zip(range(1, len(columns)+1), columns):
        plt.subplot(3, 3, i)
        print(i,column)
        print(len(df["Default"]));print(len(df[column]))
        sns.boxplot(x="Default", y=column, data=df, linewidth=0.5)
        plt.xlabel('Default')

In [15]:
df.columns

Index(['index', 'LoanID', 'Channel', 'SellerName', 'OrInterestRate', 'OrUnpaidPrinc', 'OrLoanTerm', 'OrDate', 'FirstPayment', 'OrLTV', 'OrCLTV', 'NumBorrow', 'DTIRat', 'CreditScore', 'FTHomeBuyer', 'LoanPurpose', 'PropertyType', 'NumUnits', 'OccStatus', 'PropertyState', 'Zip', 'MortInsPerc', 'ProductType', 'CoCreditScore', 'Extra', 'MortInsType', 'RelMortInd', 'Default'], dtype='object')

In [16]:
df.drop(['index','OrDate','OrLTV','MortInsPerc','RelMortInd','FirstPayment'], axis=1, inplace=True)

print(df.shape)

(11232359, 22)


In [17]:
def fillnan(df):
    if n_gpus == 0: 
        columns = df.columns[df.isnull().any().tolist()]
    else: 
        columns = df.columns[df.isnull().any().to_arrow().to_pylist()]
    for name in columns:
        print(name)
        if df[name].dtype == 'object':
            df.loc[df[name].isnull(), name] = df[name].mode().to_pandas()[0]
        else:
            #  a possible better way could be to groupby the particular time period and use those values
            #  ex. the interest rate during a particular month, year, state combination.
            df.loc[df[name].isnull(), name] = df[name].mean()
    return df
df = fillnan(df)

OrInterestRate
OrCLTV
NumBorrow
DTIRat
CreditScore
CoCreditScore
Extra


Now let's factorize the categorical columns and save the cleaned data

In [18]:
df['Channel'], Channel = df['Channel'].factorize()
df['SellerName'], SellerName = df['SellerName'].factorize()
df['FTHomeBuyer'], FTHomeBuyer = df['FTHomeBuyer'].factorize()
df['LoanPurpose'], LoanPurpose = df['LoanPurpose'].factorize()
df['PropertyType'], PropertyType = df['PropertyType'].factorize()
df['OccStatus'], OccStatus = df['OccStatus'].factorize()
df['PropertyState'], PropertyState = df['PropertyState'].factorize()
df['Zip'], Zip = df['Zip'].factorize()
df['ProductType'], ProductType = df['ProductType'].factorize()
df['MortInsType'], MortInsType = df['MortInsType'].factorize()

In [19]:
# save cleaned data to parquet file

os.makedirs('/rapids/notebooks/cleaned_data', exist_ok=True)
df.to_parquet('/rapids/notebooks/cleaned_data/fnma_cleaned.pq')

In [20]:
# save these mappings for transforming back and forth between factorized and real values
mappings = {
    'Channel': Channel.to_pandas().tolist(),
    'SellerName': SellerName.to_pandas().tolist(),
    'FTHomeBuyer': FTHomeBuyer.to_pandas().tolist(),
    'LoanPurpose': LoanPurpose.to_pandas().tolist(),
    'PropertyType': PropertyType.to_pandas().tolist(),
    'OccStatus': OccStatus.to_pandas().tolist(),
    'PropertyState': PropertyState.to_pandas().tolist(),
    'Zip': Zip.to_pandas().tolist(),
    'ProductType': ProductType.to_pandas().tolist(),
    'MortInsType': MortInsType.to_pandas().tolist(),
}

In [21]:
with open('/rapids/notebooks/cleaned_data/fnma_mappings.pkl', 'wb') as f:
    pickle.dump(mappings, f)

# #  test opening pickle file to make sure items serialized appropriately
# with open('fnma_mappings.pkl', 'rb') as f:
#     asdf = pickle.load(f)